<a href="https://colab.research.google.com/github/shiehn/dawnet-remotes/blob/main/DAWNet_Remote_BeatNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTALL DEPENDENCIES

In [ ]:
#Check if BeatNet is installed, if not then install it
try:
    from BeatNet.BeatNet import BeatNet
except ImportError:
    !pip install BeatNet
    from BeatNet.BeatNet import BeatNet

#Check if librosa is installed, if not then install it
try:
    import librosa
except ImportError:
    !pip install librosa
    import librosa

In [ ]:
# Check if ffmpeg is installed, if not then install it
ffmpeg_installed = !command -v ffmpeg
if not ffmpeg_installed:
    !apt-get install ffmpeg

#Check if dawnet-client is installed, if not then install it
try:
    import dawnet_client
except ImportError:
    !pip install dawnet-client
    import dawnet_client

try:
    import dawnet_client
except ImportError:
  !pip install dawnet-client --upgrade


import dawnet_client.core as dawnet
from dawnet_client import DAWNetFilePath

placeholder_txt = "Enter the token generated by the DAWNet plugin"
DAWNET_TOKEN = "Enter the token generated by the DAWNet plugin" #@param {type:"string"}
dawnet_token = DAWNET_TOKEN

if dawnet_token is None or dawnet_token == "" or dawnet_token == placeholder_txt:
  print("ERROR: The token provided is not valid.")
  exit()

async def dawnet_func(input_file: DAWNetFilePath):
    try:

        # -------------------------
        # ADD YOUR CUSTOM CODE HERE
        # ADD YOUR CUSTOM CODE HERE
        # ADD YOUR CUSTOM CODE HERE
        # -------------------------

        # after executing your custom code you send data back to the plugin like so ..
        await dawnet.output().add_file(input_file)
        await dawnet.output().add_message("Returning the input file")
        await dawnet.output().send()

        return True
    except Exception as e:
        await dawnet.output().add_error(f"Error in arbitrary_method: {e}")

        return False


dawnet.set_token(token=dawnet_token)
dawnet.set_name("DAWNet Template")
dawnet.set_description("This is a template for creating a DAWNet Remote")
dawnet.register_method(dawnet_func)

dawnet.connect_to_server()